<a href="https://colab.research.google.com/github/AnanthSrinivasan/TickerFetcherDemo/blob/main/screener_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------
# Imports & Setup
# ----------------------------
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time
import random

FINVIZ_BASE = "https://finviz.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}

session = requests.Session()
session.headers.update(HEADERS)

# ----------------------------
# Part 1: Screener Fetch & Save
# ----------------------------
screener_urls = {
    "10% Change": (
        f"{FINVIZ_BASE}/screener.ashx?v=151"
        f"&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,"
        f"ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&"
        f"c=0,1,2,3,4,5,6,64,67,65,66"
    ),
    "Growth": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,"
        f"ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,"
        f"ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4"
    ),
    "IPO": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_midover,ind_stocksonly,ipodate_prev3yrs,sh_avgvol_o1000,"
        f"sh_price_o10,ta_beta_o0.5,ta_sma20_pa&ft=4"
    ),
    "52 Week High": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_beta_o1,"
        f"ta_highlow52w_nh&ft=4"
    ),
    "Week 20%+ Gain": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_smallover,ind_stocksonly,sh_avgvol_o1000,ta_perf_1w30o,"
        f"ta_sma20_pa,ta_volatility_wo4&ft=4&o=-marketcap"
    )
}

def fetch_all_tickers(screener_url: str, max_pages: int = 10) -> pd.DataFrame:
    combined = []
    seen = set()
    page = 1

    while page <= max_pages:
        resp = session.get(f"{screener_url}&r={1+(page-1)*20}", timeout=10)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        rows = soup.select('tr[valign="top"]')
        if not rows:
            break

        new_data = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 11:
                ticker = cols[1].text.strip()
                if ticker and ticker not in seen:
                    combined.append([c.text.strip() for c in cols])
                    seen.add(ticker)
                    new_data = True
        if not new_data:
            break
        page += 1
        time.sleep(1)

    columns = ['No.', 'Ticker', 'Company', 'Sector', 'Industry',
               'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change']
    return pd.DataFrame(combined, columns=columns) if combined else pd.DataFrame(columns=columns)

def aggregate_and_save(screener_map: dict) -> (pd.DataFrame, str, str):
    mapping = defaultdict(list)
    today = datetime.date.today().strftime("%Y-%m-%d")

    for name, url in screener_map.items():
        df = fetch_all_tickers(url)
        print(f"{name}: {len(df)} tickers found")
        for t in df['Ticker'].unique():
            mapping[t].append(name)

    if not mapping:
        mapping['TSLA'].append('Default Screener')

    data = []
    for t, screens in mapping.items():
        data.append({
            'Ticker': t,
            'Appearances': len(screens),
            'Screeners': ", ".join(screens)
        })
    summary_df = pd.DataFrame(data).sort_values(['Appearances','Ticker'], ascending=[False, True])

    csv_file = f"finviz_screeners_{today}.csv"
    html_file = f"finviz_screeners_{today}.html"
    summary_df.to_csv(csv_file, index=False)
    summary_df.to_html(html_file, index=False)

    return summary_df, csv_file, html_file

# ----------------------------
# Part 2: Snapshot Fetch with Retries
# ----------------------------
def get_snapshot_metrics(ticker: str, max_retries: int = 5):
    for attempt in range(max_retries):
        try:
            resp = session.get(f"{FINVIZ_BASE}/quote.ashx", params={"t": ticker})
            resp.raise_for_status()
            soup = BeautifulSoup(resp.content, "html.parser")
            table = soup.find("table", class_="snapshot-table2")
            if not table:
                raise ValueError("Snapshot table not found")

            data = {}
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                for key_cell, val_cell in zip(cells[0::2], cells[1::2]):
                    key = key_cell.get_text(strip=True).rstrip('.')
                    data[key] = val_cell.get_text(strip=True)

            atr_pct = float(data.get("ATR (14)", 0)) / float(data.get("Price", "1").replace(',', '')) * 100
            eps_str = data.get("EPS Y/Y TTM", '0').replace('%','').strip()
            eps = float(eps_str) if eps_str != '-' else 0.0

            sales_str = data.get("Sales Y/Y TTM", '0').replace('%','').strip()
            sales = float(sales_str) if sales_str != '-' else 0.0
            return atr_pct, eps, sales
        except requests.HTTPError as e:
            if e.response.status_code == 429:
                wait = (2 ** attempt) + random.random()
                print(f"Rate limited fetching snapshot for {ticker}; retrying in {wait:.1f}s…")
                time.sleep(wait)
                continue
            else:
                print(f"HTTP error for {ticker}: {e}")
                break
        except Exception as e:
            print(f"Error fetching snapshot for {ticker}: {e}")
            break

    return None, None, None

# ----------------------------
# Part 3: Chart Gallery
# ----------------------------
def generate_finviz_gallery(tickers: list) -> str:
    today = datetime.date.today().strftime("%Y-%m-%d")
    out_html = f"finviz_chart_grid_{today}.html"
    html = [
        '<html><head><title>Finviz Chart Gallery</title>',
        '<style>',
        'body { font-family: Arial; background: #f5f5f5; padding: 20px }',
        '.chart-grid { display: flex; flex-wrap: wrap; gap: 20px }',
        '.chart-item { width: 23%; background: white; border:1px solid #ccc;',
        'padding:10px; box-shadow:2px 2px 5px rgba(0,0,0,0.1); text-align:center }',
        '.chart-item img { max-width:100%; height:auto }',
        'h2 { text-align:center }',
        '</style></head><body><h2>Finviz Chart Gallery</h2>',
        '<div class="chart-grid">'
    ]
    for t in tickers:
        url = f"{FINVIZ_BASE}/chart.ashx"
        params = {"t": t, "ty": "c", "ta": 1, "p": "d", "s": "m"}
        req = requests.Request('GET', url, params=params).prepare()
        html.append(f'<div class="chart-item"><h4>{t}</h4>'
                    f'<img src="{req.url}" alt="{t}"></div>')
    html.append('</div></body></html>')

    with open(out_html, 'w') as f:
        f.write("\n".join(html))
    return out_html

# ----------------------------
# Part 4: Main Execution with Logging
# ----------------------------
if __name__ == "__main__":
    summary_df, csv_path, html_summary = aggregate_and_save(screener_urls)
    print(f"Summary CSV: {csv_path}\nSummary HTML: {html_summary}")

    print(f"Initial tickers count: {len(summary_df)}")

    # Fetch snapshot metrics
    summary_df[['ATR%', 'EPS Y/Y TTM', 'Sales Y/Y TTM']] = summary_df['Ticker'].apply(
        lambda t: pd.Series(get_snapshot_metrics(t))
    )

    filter_a = summary_df[summary_df['ATR%'] > 3.0]
    print(f"Tickers with ATR% > 3.0: {len(filter_a)}")

    # Ask user whether to apply EPS and Sales filters
    apply_earnings_filter = input("Filter further by EPS > 20 and Sales > 20? (y/n): ").strip().lower()
    if apply_earnings_filter == 'y':
        final_filtered = filter_a[
            (filter_a['EPS Y/Y TTM'] > 20) &
            (filter_a['Sales Y/Y TTM'] > 20)
        ]
        print(f"Tickers with ATR% > 3.0 AND EPS & Sales Y/Y TTM > 20: {len(final_filtered)}")
    else:
        final_filtered = filter_a
        print("Skipped EPS/Sales filtering. Using ATR% > 3.0 only.")

    today = datetime.date.today().strftime("%Y-%m-%d")
    out_file = f"finviz_filtered_{today}.csv"
    final_filtered.to_csv(out_file, index=False)
    print(f"Filtered results saved: {out_file}")

    gallery_path = generate_finviz_gallery(final_filtered['Ticker'].tolist())
    print(f"Filtered chart gallery HTML: {gallery_path}")

10% Change: 9 tickers found
Growth: 44 tickers found
IPO: 10 tickers found
52 Week High: 7 tickers found
Week 20%+ Gain: 0 tickers found
Summary CSV: finviz_screeners_2025-08-03.csv
Summary HTML: finviz_screeners_2025-08-03.html
Initial tickers count: 63
Rate limited fetching snapshot for BSX; retrying in 1.4s…
Rate limited fetching snapshot for NI; retrying in 1.3s…
Rate limited fetching snapshot for WK; retrying in 1.0s…
Tickers with ATR% > 3.0: 42
Filter further by EPS > 20 and Sales > 20? (y/n): n
Skipped EPS/Sales filtering. Using ATR% > 3.0 only.
Filtered results saved: finviz_filtered_2025-08-03.csv
Filtered chart gallery HTML: finviz_chart_grid_2025-08-03.html


In [ ]:
# ----------------------------
# Imports & Setup
# ----------------------------
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time
import random

FINVIZ_BASE = "https://finviz.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}

session = requests.Session()
session.headers.update(HEADERS)

# ----------------------------
# Part 1: Screener Fetch & Save
# ----------------------------
screener_urls = {
    "10% Change": (
        f"{FINVIZ_BASE}/screener.ashx?v=151"
        f"&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,"
        f"ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&"
        f"c=0,1,2,3,4,5,6,64,67,65,66"
    ),
    "Growth": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,"
        f"ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,"
        f"ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4"
    ),
    "IPO": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_midover,ind_stocksonly,ipodate_prev3yrs,sh_avgvol_o1000,"
        f"sh_price_o10,ta_beta_o0.5,ta_sma20_pa&ft=4"
    ),
    "52 Week High": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_beta_o1,"
        f"ta_highlow52w_nh&ft=4"
    ),
    "Week 20%+ Gain": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_smallover,ind_stocksonly,sh_avgvol_o1000,ta_perf_1w30o,"
        f"ta_sma20_pa,ta_volatility_wo4&ft=4&o=-marketcap"
    )
}

def fetch_all_tickers(screener_url: str, max_pages: int = 10) -> pd.DataFrame:
    combined = []
    seen = set()
    page = 1

    while page <= max_pages:
        resp = session.get(f"{screener_url}&r={1+(page-1)*20}", timeout=10)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        rows = soup.select('tr[valign="top"]')
        if not rows:
            break

        new_data = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 11:
                ticker = cols[1].text.strip()
                if ticker and ticker not in seen:
                    combined.append([c.text.strip() for c in cols])
                    seen.add(ticker)
                    new_data = True
        if not new_data:
            break
        page += 1
        time.sleep(1)

    columns = ['No.', 'Ticker', 'Company', 'Sector', 'Industry',
               'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change']
    return pd.DataFrame(combined, columns=columns) if combined else pd.DataFrame(columns=columns)

def aggregate_and_save(screener_map: dict) -> (pd.DataFrame, str, str):
    mapping = defaultdict(list)
    today = datetime.date.today().strftime("%Y-%m-%d")

    for name, url in screener_map.items():
        df = fetch_all_tickers(url)
        print(f"{name}: {len(df)} tickers found")
        for t in df['Ticker'].unique():
            mapping[t].append(name)

    if not mapping:
        mapping['TSLA'].append('Default Screener')

    data = []
    for t, screens in mapping.items():
        data.append({
            'Ticker': t,
            'Appearances': len(screens),
            'Screeners': ", ".join(screens)
        })
    summary_df = pd.DataFrame(data).sort_values(['Appearances','Ticker'], ascending=[False, True])

    csv_file = f"finviz_screeners_{today}.csv"
    html_file = f"finviz_screeners_{today}.html"
    summary_df.to_csv(csv_file, index=False)
    summary_df.to_html(html_file, index=False)

    return summary_df, csv_file, html_file

# ----------------------------
# Part 2: Snapshot Fetch with Retries
# ----------------------------
def get_snapshot_metrics(ticker: str, max_retries: int = 5):
    for attempt in range(max_retries):
        try:
            resp = session.get(f"{FINVIZ_BASE}/quote.ashx", params={"t": ticker})
            resp.raise_for_status()
            soup = BeautifulSoup(resp.content, "html.parser")
            table = soup.find("table", class_="snapshot-table2")
            if not table:
                raise ValueError("Snapshot table not found")

            data = {}
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                for key_cell, val_cell in zip(cells[0::2], cells[1::2]):
                    key = key_cell.get_text(strip=True).rstrip('.')
                    data[key] = val_cell.get_text(strip=True)

            atr_pct = float(data.get("ATR (14)", 0)) / float(data.get("Price", "1").replace(',', '')) * 100
            eps = float(data.get("EPS Y/Y TTM", '0').replace('%',''))
            sales = float(data.get("Sales Y/Y TTM", '0').replace('%',''))
            return atr_pct, eps, sales

        except requests.HTTPError as e:
            if e.response.status_code == 429:
                wait = (2 ** attempt) + random.random()
                print(f"Rate limited fetching snapshot for {ticker}; retrying in {wait:.1f}s…")
                time.sleep(wait)
                continue
            else:
                print(f"HTTP error for {ticker}: {e}")
                break
        except Exception as e:
            print(f"Error fetching snapshot for {ticker}: {e}")
            break

    return None, None, None

# ----------------------------
# Part 3: Chart Gallery
# ----------------------------
def generate_finviz_gallery(tickers: list) -> str:
    today = datetime.date.today().strftime("%Y-%m-%d")
    out_html = f"finviz_chart_grid_{today}.html"
    html = [
        '<html><head><title>Finviz Chart Gallery</title>',
        '<style>',
        'body { font-family: Arial; background: #f5f5f5; padding: 20px }',
        '.chart-grid { display: flex; flex-wrap: wrap; gap: 20px }',
        '.chart-item { width: 23%; background: white; border:1px solid #ccc;',
        'padding:10px; box-shadow:2px 2px 5px rgba(0,0,0,0.1); text-align:center }',
        '.chart-item img { max-width:100%; height:auto }',
        'h2 { text-align:center }',
        '</style></head><body><h2>Finviz Chart Gallery</h2>',
        '<div class="chart-grid">'
    ]
    for t in tickers:
        url = f"{FINVIZ_BASE}/chart.ashx"
        params = {"t": t, "ty": "c", "ta": 1, "p": "d", "s": "m"}
        req = requests.Request('GET', url, params=params).prepare()
        html.append(f'<div class="chart-item"><h4>{t}</h4>'
                    f'<img src="{req.url}" alt="{t}"></div>')
    html.append('</div></body></html>')

    with open(out_html, 'w') as f:
        f.write("\n".join(html))
    return out_html

# ----------------------------
# Part 4: Main Execution with Logging
# ----------------------------
if __name__ == "__main__":
    summary_df, csv_path, html_summary = aggregate_and_save(screener_urls)
    print(f"Summary CSV: {csv_path}\nSummary HTML: {html_summary}")

    print(f"Initial tickers count: {len(summary_df)}")

    # Fetch snapshot metrics
    summary_df[['ATR%', 'EPS Y/Y TTM', 'Sales Y/Y TTM']] = summary_df['Ticker'].apply(
        lambda t: pd.Series(get_snapshot_metrics(t))
    )

    filter_a = summary_df[summary_df['ATR%'] > 3.0]
    print(f"Tickers with ATR% > 3.0: {len(filter_a)}")

    # filter_b = filter_a[
    #     (filter_a['EPS Y/Y TTM'] > 20) &
    #     (filter_a['Sales Y/Y TTM'] > 20)
    # ]
    # print(f"Tickers with ATR% > 3.0 AND EPS & Sales Y/Y TTM > 20: {len(filter_b)}")

    # today = datetime.date.today().strftime("%Y-%m-%d")
    # filter_b.to_csv(f"finviz_filtered_{today}.csv", index=False)
    # print(f"Filtered results saved: finviz_filtered_{today}.csv")

    gallery_path = generate_finviz_gallery(filter_a['Ticker'].tolist())
    print(f"Filtered chart gallery HTML: {gallery_path}")


10% Change: 10 tickers found
Growth: 42 tickers found
IPO: 26 tickers found
52 Week High: 32 tickers found
Week 20%+ Gain: 56 tickers found
Summary CSV: finviz_screeners_2025-05-14.csv
Summary HTML: finviz_screeners_2025-05-14.html
Initial tickers count: 143
Error fetching snapshot for ACHR: could not convert string to float: '-'
Error fetching snapshot for AHR: could not convert string to float: '-'
Rate limited fetching snapshot for AAOI; retrying in 1.3s…
Rate limited fetching snapshot for BCRX; retrying in 1.4s…
Rate limited fetching snapshot for BMBL; retrying in 1.3s…
Error fetching snapshot for BTDR: could not convert string to float: '-'
Rate limited fetching snapshot for ELAN; retrying in 2.0s…
Rate limited fetching snapshot for HSAI; retrying in 1.3s…
Rate limited fetching snapshot for NXT; retrying in 1.4s…
Error fetching snapshot for RYET: could not convert string to float: '-'
Rate limited fetching snapshot for SHLS; retrying in 1.6s…
Error fetching snapshot for SOC: could

In [ ]:
# ----------------------------
# Imports & Setup
# ----------------------------
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time
import random

FINVIZ_BASE = "https://finviz.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}

session = requests.Session()
session.headers.update(HEADERS)

# ----------------------------
# Part 1: Screener Fetch & Save
# ----------------------------
screener_urls = {
    "10% Change": (
        f"{FINVIZ_BASE}/screener.ashx?v=151"
        f"&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,"
        f"ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&"
        f"c=0,1,2,3,4,5,6,64,67,65,66"
    ),
    "Growth": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,"
        f"ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,"
        f"ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4"
    ),
    "IPO": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_midover,ind_stocksonly,ipodate_prev3yrs,sh_avgvol_o1000,"
        f"sh_price_o10,ta_beta_o0.5,ta_sma20_pa&ft=4"
    ),
    "52 Week High": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_beta_o1,"
        f"ta_highlow52w_nh&ft=4"
    ),
    "Week 20%+ Gain": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_smallover,ind_stocksonly,sh_avgvol_o1000,ta_perf_1w30o,"
        f"ta_sma20_pa,ta_volatility_wo4&ft=4&o=-marketcap"
    )
}

def fetch_all_tickers(screener_url: str, max_pages: int = 10) -> pd.DataFrame:
    combined = []
    seen = set()
    page = 1

    while page <= max_pages:
        resp = session.get(f"{screener_url}&r={1+(page-1)*20}", timeout=10)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        rows = soup.select('tr[valign="top"]')
        if not rows:
            break

        new_data = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 11:
                ticker = cols[1].text.strip()
                if ticker and ticker not in seen:
                    combined.append([c.text.strip() for c in cols])
                    seen.add(ticker)
                    new_data = True
        if not new_data:
            break
        page += 1
        time.sleep(1)

    columns = ['No.', 'Ticker', 'Company', 'Sector', 'Industry',
               'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change']
    return pd.DataFrame(combined, columns=columns) if combined else pd.DataFrame(columns=columns)

def aggregate_and_save(screener_map: dict) -> (pd.DataFrame, str, str):
    mapping = defaultdict(list)
    today = datetime.date.today().strftime("%Y-%m-%d")

    for name, url in screener_map.items():
        df = fetch_all_tickers(url)
        print(f"{name}: {len(df)} tickers found")
        for t in df['Ticker'].unique():
            mapping[t].append(name)

    if not mapping:
        mapping['TSLA'].append('Default Screener')

    data = []
    for t, screens in mapping.items():
        data.append({
            'Ticker': t,
            'Appearances': len(screens),
            'Screeners': ", ".join(screens)
        })
    summary_df = pd.DataFrame(data).sort_values(['Appearances','Ticker'], ascending=[False, True])

    csv_file = f"finviz_screeners_{today}.csv"
    html_file = f"finviz_screeners_{today}.html"
    summary_df.to_csv(csv_file, index=False)
    summary_df.to_html(html_file, index=False)

    return summary_df, csv_file, html_file

# ----------------------------
# Part 2: Snapshot Fetch with Retries
# ----------------------------
def get_snapshot_metrics(ticker: str, max_retries: int = 5):
    for attempt in range(max_retries):
        try:
            resp = session.get(f"{FINVIZ_BASE}/quote.ashx", params={"t": ticker})
            resp.raise_for_status()
            soup = BeautifulSoup(resp.content, "html.parser")
            table = soup.find("table", class_="snapshot-table2")
            if not table:
                raise ValueError("Snapshot table not found")

            data = {}
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                for key_cell, val_cell in zip(cells[0::2], cells[1::2]):
                    key = key_cell.get_text(strip=True).rstrip('.')
                    data[key] = val_cell.get_text(strip=True)

            atr_pct = float(data.get("ATR (14)", 0)) / float(data.get("Price", "1").replace(',', '')) * 100
            eps = float(data.get("EPS Y/Y TTM", '0').replace('%',''))
            sales = float(data.get("Sales Y/Y TTM", '0').replace('%',''))
            return atr_pct, eps, sales

        except requests.HTTPError as e:
            if e.response.status_code == 429:
                wait = (2 ** attempt) + random.random()
                print(f"Rate limited fetching snapshot for {ticker}; retrying in {wait:.1f}s…")
                time.sleep(wait)
                continue
            else:
                print(f"HTTP error for {ticker}: {e}")
                break
        except Exception as e:
            print(f"Error fetching snapshot for {ticker}: {e}")
            break

    return None, None, None

# ----------------------------
# Part 3: Chart Gallery
# ----------------------------
def generate_finviz_gallery(tickers: list) -> str:
    today = datetime.date.today().strftime("%Y-%m-%d")
    out_html = f"finviz_chart_grid_{today}.html"
    html = [
        '<html><head><title>Finviz Chart Gallery</title>',
        '<style>',
        'body { font-family: Arial; background: #f5f5f5; padding: 20px }',
        '.chart-grid { display: flex; flex-wrap: wrap; gap: 20px }',
        '.chart-item { width: 23%; background: white; border:1px solid #ccc;',
        'padding:10px; box-shadow:2px 2px 5px rgba(0,0,0,0.1); text-align:center }',
        '.chart-item img { max-width:100%; height:auto }',
        'h2 { text-align:center }',
        '</style></head><body><h2>Finviz Chart Gallery</h2>',
        '<div class="chart-grid">'
    ]
    for t in tickers:
        url = f"{FINVIZ_BASE}/chart.ashx"
        params = {"t": t, "ty": "c", "ta": 1, "p": "d", "s": "m"}
        req = requests.Request('GET', url, params=params).prepare()
        html.append(f'<div class="chart-item"><h4>{t}</h4>'
                    f'<img src="{req.url}" alt="{t}"></div>')
    html.append('</div></body></html>')

    with open(out_html, 'w') as f:
        f.write("\n".join(html))
    return out_html

# ----------------------------
# Part 4: Main Execution with Logging
# ----------------------------
if __name__ == "__main__":
    summary_df, csv_path, html_summary = aggregate_and_save(screener_urls)
    print(f"Summary CSV: {csv_path}\nSummary HTML: {html_summary}")

    print(f"Initial tickers count: {len(summary_df)}")

    # Fetch snapshot metrics
    summary_df[['ATR%', 'EPS Y/Y TTM', 'Sales Y/Y TTM']] = summary_df['Ticker'].apply(
        lambda t: pd.Series(get_snapshot_metrics(t))
    )

    filter_a = summary_df[summary_df['ATR%'] > 3.0]
    print(f"Tickers with ATR% > 3.0: {len(filter_a)}")

    filter_b = filter_a[
        (filter_a['EPS Y/Y TTM'] > 20) &
        (filter_a['Sales Y/Y TTM'] > 20)
    ]
    print(f"Tickers with ATR% > 3.0 AND EPS & Sales Y/Y TTM > 20: {len(filter_b)}")

    today = datetime.date.today().strftime("%Y-%m-%d")
    filter_b.to_csv(f"finviz_filtered_{today}.csv", index=False)
    print(f"Filtered results saved: finviz_filtered_{today}.csv")

    gallery_path = generate_finviz_gallery(filter_b['Ticker'].tolist())
    print(f"Filtered chart gallery HTML: {gallery_path}")


10% Change: 10 tickers found
Growth: 42 tickers found
IPO: 26 tickers found
52 Week High: 32 tickers found
Week 20%+ Gain: 56 tickers found
Summary CSV: finviz_screeners_2025-05-14.csv
Summary HTML: finviz_screeners_2025-05-14.html
Initial tickers count: 143
Error fetching snapshot for ACHR: could not convert string to float: '-'
Error fetching snapshot for AHR: could not convert string to float: '-'
Rate limited fetching snapshot for AAOI; retrying in 1.5s…
Rate limited fetching snapshot for BROS; retrying in 1.1s…
Rate limited fetching snapshot for BROS; retrying in 2.1s…
Error fetching snapshot for BTDR: could not convert string to float: '-'
Rate limited fetching snapshot for ENVX; retrying in 1.7s…
Rate limited fetching snapshot for JCI; retrying in 1.3s…
Rate limited fetching snapshot for JCI; retrying in 2.3s…
Rate limited fetching snapshot for RUM; retrying in 1.9s…
Error fetching snapshot for RYET: could not convert string to float: '-'
Error fetching snapshot for SOC: could n

In [ ]:
%%time

import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time
import random

# ----------------------------
# Constants & Shared Session
# ----------------------------
FINVIZ_BASE = "https://finviz.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}

session = requests.Session()
session.headers.update(HEADERS)

# ----------------------------
# Part 1: Screener Fetch & Save
# ----------------------------

screener_urls = {
    "10% Change": (
        f"{FINVIZ_BASE}/screener.ashx?v=151"
        f"&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,"
        f"ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&"
        f"c=0,1,2,3,4,5,6,64,67,65,66"
    ),
    "Growth": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,"
        f"ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,"
        f"ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4"
    ),
    "IPO": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,"
        f"ta_sma20_pa&ft=4"
    ),
    "52 Week High": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4"
    ),
    "Week 20%+ Gain": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,"
        f"ta_volatility_wo4&ft=4&o=-marketcap&r=25"
    )
}


def fetch_all_tickers(screener_url: str, max_pages: int = 10) -> pd.DataFrame:
    combined = []
    seen = set()
    page = 1

    while page <= max_pages:
        resp = session.get(f"{screener_url}&r={1+(page-1)*20}", timeout=10)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        rows = soup.select('tr[valign="top"]')
        if not rows:
            break

        new_data = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 11:
                ticker = cols[1].text.strip()
                if ticker and ticker not in seen:
                    combined.append([c.text.strip() for c in cols])
                    seen.add(ticker)
                    new_data = True
        if not new_data:
            break
        page += 1
        time.sleep(1)

    columns = ['No.', 'Ticker', 'Company', 'Sector', 'Industry',
               'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change']
    return pd.DataFrame(combined, columns=columns) if combined else pd.DataFrame(columns=columns)


def aggregate_and_save(screener_map: dict) -> (pd.DataFrame, str, str):
    mapping = defaultdict(list)
    today = datetime.date.today().strftime("%Y-%m-%d")

    for name, url in screener_map.items():
        df = fetch_all_tickers(url)
        print(f"{name}: {len(df)} tickers found")
        for t in df['Ticker'].unique():
            mapping[t].append(name)

    if not mapping:
        mapping['TSLA'].append('Default Screener')

    data = []
    for t, screens in mapping.items():
        data.append({
            'Ticker': t,
            'Appearances': len(screens),
            'Screeners': ", ".join(screens)
        })
    summary_df = pd.DataFrame(data).sort_values(['Appearances','Ticker'], ascending=[False, True])

    csv_file = f"finviz_screeners_{today}.csv"
    html_file = f"finviz_screeners_{today}.html"
    summary_df.to_csv(csv_file, index=False)
    summary_df.to_html(html_file, index=False)

    return summary_df, csv_file, html_file

# ----------------------------
# Part 2: ATR% Scraper & Filter
# ----------------------------

def get_atr_pct(ticker: str, max_retries: int = 5) -> float:
    """
    Fetches ATR% using current Price field with retries on HTTP 429.
    """
    for attempt in range(max_retries):
        try:
            resp = session.get(f"{FINVIZ_BASE}/quote.ashx", params={"t": ticker})
            resp.raise_for_status()
            soup = BeautifulSoup(resp.content, "html.parser")
            table = soup.find("table", class_="snapshot-table2")
            if not table:
                raise ValueError("Snapshot table not found")

            data = {}
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                for key_cell, val_cell in zip(cells[0::2], cells[1::2]):
                    key = key_cell.get_text(strip=True).rstrip('.')
                    data[key] = val_cell.get_text(strip=True)

            atr_str   = data.get("ATR (14)")
            price_str = data.get("Price")
            eps_str = data.get("EPS Y/Y TTM")
            sales_str = data.get("Sales Y/Y TTM")

            if not atr_str or not price_str:
                raise ValueError("Missing ATR or Price")

            atr   = float(atr_str)
            price = float(price_str.replace(',',''))
            eps = float(eps_str)
            sales = float(sales_str)
            return (atr / price) * 100

        except requests.HTTPError as e:
            if e.response.status_code == 429:
                wait = (2 ** attempt) + random.random()
                print(f"Rate limited fetching ATR% for {ticker}; retrying in {wait:.1f}s…")
                time.sleep(wait)
                continue
            else:
                print(f"HTTP error for {ticker}: {e}")
                break
        except Exception as e:
            print(f"Error parsing ATR% for {ticker}: {e}")
            break

    print(f"⚠️ Giving up on ATR% for {ticker}")
    return None

# ----------------------------
# Part 3: Generate Chart Gallery with Date
# ----------------------------

def generate_finviz_gallery(tickers: list) -> str:
    today = datetime.date.today().strftime("%Y-%m-%d")
    out_html = f"finviz_chart_grid_{today}.html"
    html = [
        '<html><head><title>Finviz Chart Gallery</title>',
        '<style>',
        'body { font-family: Arial; background: #f5f5f5; padding: 20px }',
        '.chart-grid { display: flex; flex-wrap: wrap; gap: 20px }',
        '.chart-item { width: 23%; background: white; border:1px solid #ccc;',
        'padding:10px; box-shadow:2px 2px 5px rgba(0,0,0,0.1); text-align:center }',
        '.chart-item img { max-width:100%; height:auto }',
        'h2 { text-align:center }',
        '</style></head><body><h2>Finviz Chart Gallery</h2>',
        '<div class="chart-grid">'
    ]
    for t in tickers:
        url = f"{FINVIZ_BASE}/chart.ashx"
        params = {"t": t, "ty": "c", "ta": 1, "p": "d", "s": "m"}
        req = requests.Request('GET', url, params=params).prepare()
        html.append(f'<div class="chart-item"><h4>{t}</h4>'
                    f'<img src="{req.url}" alt="{t}"></div>')
    html.append('</div></body></html>')

    with open(out_html, 'w') as f:
        f.write("\n".join(html))
    return out_html

# ----------------------------
# Main Execution
# ----------------------------
if __name__ == "__main__":
    # Part 1: Summary
    summary_df, csv_path, html_summary = aggregate_and_save(screener_urls)
    print(f"Summary CSV: {csv_path}\nSummary HTML: {html_summary}")

    # Part 2: ATR% & Filtering
    total = len(summary_df)
    summary_df['ATR%'] = summary_df['Ticker'].apply(get_atr_pct)
    available = summary_df['ATR%'].notnull().sum()
    filtered = summary_df[summary_df['ATR%'] > 3.0]
    print(f"Total tickers: {total}")
    print(f"Tickers with ATR% data: {available}")
    print(f"Tickers > 3% ATR%: {len(filtered)}")

    today = datetime.date.today().strftime("%Y-%m-%d")
    filtered.to_csv(f"finviz_screeners_filtered_{today}.csv", index=False)
    filtered.to_html(f"finviz_screeners_filtered_{today}.html", index=False)
    print(f"Filtered CSV/HTML written for ATR% > 3%")

    # Part 3: Gallery for filtered tickers
    gallery_path = generate_finviz_gallery(filtered['Ticker'].tolist())
    print(f"Filtered chart gallery HTML: {gallery_path}")

10% Change: 13 tickers found
Growth: 34 tickers found
IPO: 46 tickers found
52 Week High: 11 tickers found
Week 20%+ Gain: 37 tickers found
Summary CSV: finviz_screeners_2025-05-11.csv
Summary HTML: finviz_screeners_2025-05-11.html
Rate limited fetching ATR% for ARM; retrying in 1.7s…
Rate limited fetching ATR% for CRWV; retrying in 1.6s…
Rate limited fetching ATR% for GOGO; retrying in 1.5s…
Rate limited fetching ATR% for OS; retrying in 1.0s…
Rate limited fetching ATR% for SNDK; retrying in 1.6s…
Total tickers: 125
Tickers with ATR% data: 125
Tickers > 3% ATR%: 105
Filtered CSV/HTML written for ATR% > 3%
Filtered chart gallery HTML: finviz_chart_grid_2025-05-11.html
CPU times: user 20.3 s, sys: 148 ms, total: 20.5 s
Wall time: 53.7 s


In [ ]:
%%time

import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time

# ----------------------------
# Constants & Shared Session
# ----------------------------
FINVIZ_BASE = "https://finviz.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}

session = requests.Session()
session.headers.update(HEADERS)

# ----------------------------
# Part 1: Screener Fetch & Save
# ----------------------------

# Screener URLs
screener_urls = {
    "10% Change": (
        f"{FINVIZ_BASE}/screener.ashx?v=151"
        f"&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,"
        f"ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&"
        f"c=0,1,2,3,4,5,6,64,67,65,66"
    ),
    "Growth": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,"
        f"ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,"
        f"ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4"
    ),
    "IPO": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,"
        f"ta_sma20_pa&ft=4"
    ),
    "52 Week High": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4"
    ),
    "Week 20%+ Gain": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,"
        f"ta_volatility_wo4&ft=4&o=-marketcap&r=25"
    )
}

def fetch_all_tickers(screener_url: str, max_pages: int = 10) -> pd.DataFrame:
    """
    Fetch tickers from a Finviz screener URL, paginating up to max_pages.
    Returns DataFrame with columns: No., Ticker, Company, Sector, Industry,
    Country, Market Cap, P/E, Volume, Price, Change.
    """
    combined = []
    seen = set()
    page = 1

    while page <= max_pages:
        resp = session.get(f"{screener_url}&r={1+(page-1)*20}", timeout=10)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        rows = soup.select('tr[valign="top"]')
        if not rows:
            break

        new_data = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 11:
                ticker = cols[1].text.strip()
                if ticker and ticker not in seen:
                    combined.append([c.text.strip() for c in cols])
                    seen.add(ticker)
                    new_data = True
        if not new_data:
            break
        page += 1
        time.sleep(1)

    columns = ['No.', 'Ticker', 'Company', 'Sector', 'Industry',
               'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change']
    return pd.DataFrame(combined, columns=columns) if combined else pd.DataFrame(columns=columns)


def aggregate_and_save(screener_map: dict) -> (pd.DataFrame, str, str):
    """
    Aggregates tickers across screeners, saves CSV & HTML summary.
    Returns (DataFrame, csv_path, html_path).
    """
    mapping = defaultdict(list)
    today = datetime.date.today().strftime("%Y-%m-%d")

    for name, url in screener_map.items():
        df = fetch_all_tickers(url)
        print(f"{name}: {len(df)} tickers found")
        for t in df['Ticker'].unique():
            mapping[t].append(name)

    if not mapping:
        mapping['TSLA'].append('Default Screener')

    data = []
    for t, screens in mapping.items():
        data.append({
            'Ticker': t,
            'Appearances': len(screens),
            'Screeners': ", ".join(screens)
        })
    summary_df = pd.DataFrame(data).sort_values(['Appearances','Ticker'], ascending=[False, True])

    csv_file = f"finviz_screeners_{today}.csv"
    html_file = f"finviz_screeners_{today}.html"
    summary_df.to_csv(csv_file, index=False)
    summary_df.to_html(html_file, index=False)

    return summary_df, csv_file, html_file

# ----------------------------
# Part 2: ATR% Scraper & Filter
# ----------------------------

def get_atr_pct(ticker: str, max_retries: int = 3) -> float:
    """
    Scrapes Finviz for ATR (14) and Prev Close, returns ATR% or None.
    Retries on 429 errors with exponential back-off.
    """
    for attempt in range(max_retries):
        try:
            resp = session.get(f"{FINVIZ_BASE}/quote.ashx", params={"t": ticker}, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.content, "html.parser")
            table = soup.find("table", class_="snapshot-table2")
            if not table:
                return None
            data = {}
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                for key_cell, val_cell in zip(cells[0::2], cells[1::2]):
                    data[key_cell.get_text(strip=True)] = val_cell.get_text(strip=True)

            atr_str   = data.get("ATR")
            price_str = data.get("Price")  # use current Price field
            if not atr_str or not price_str:
                raise ValueError("Missing ATR or Price")

            atr   = float(atr_str)
            price = float(price_str.replace(',',''))
            return (atr / price) * 100


        except requests.HTTPError as e:
            if resp.status_code == 429:
                wait = 2 ** attempt
                print(f"Rate limited fetching ATR% for {ticker}; retrying in {wait}s...")
                time.sleep(wait)
                continue
            return None
        except Exception:
            return None
    print(f"Skipping {ticker} after {max_retries} retries due to rate limits.")
    return None

# ----------------------------
# Part 3: Chart Gallery
# ----------------------------

def generate_finviz_gallery(tickers: list) -> str:
    """
    Creates HTML displaying a 4-col grid of Finviz charts, dated filename.
    """
    today = datetime.date.today().strftime("%Y-%m-%d")
    out_html = f"finviz_chart_grid_{today}.html"
    html = [
        '<html><head><title>Finviz Chart Gallery</title>',
        '<style>',
        'body { font-family: Arial; background: #f5f5f5; padding: 20px }',
        '.chart-grid { display: flex; flex-wrap: wrap; gap: 20px }',
        '.chart-item { width: 23%; background: white; border:1px solid #ccc; ',
        'padding:10px; box-shadow:2px 2px 5px rgba(0,0,0,0.1); text-align:center }',
        '.chart-item img { max-width:100%; height:auto }',
        'h2 { text-align:center }',
        '</style></head><body><h2>Finviz Chart Gallery</h2>',
        '<div class="chart-grid">'
    ]
    for t in tickers:
        params = {'t': t, 'ty': 'c', 'ta': '1', 'p': 'd', 's': 'm'}
        img_src = requests.Request('GET', f"{FINVIZ_BASE}/chart.ashx", params=params).prepare().url
        html.append(f'<div class="chart-item"><h4>{t}</h4>'
                    f'<img src="{img_src}" alt="{t}"></div>')
    html.append('</div></body></html>')
    with open(out_html, 'w') as f:
        f.write("\n".join(html))
    return out_html

# ----------------------------
# Main Execution
# ----------------------------
if __name__ == "__main__":
    # 1. Aggregate screeners
    summary_df, csv_path, html_summary = aggregate_and_save(screener_urls)
    total = len(summary_df)
    print(f"Total tickers before ATR% filter: {total}")

    # 2. Compute ATR% and filter >3%
    summary_df['ATR%'] = summary_df['Ticker'].apply(lambda t: get_atr_pct(t))
    filtered_df = summary_df[summary_df['ATR%'] > 3.0]
    filtered_count = len(filtered_df)
    print(f"Tickers with ATR% > 3%: {filtered_count}")

    # 3. Save filtered results
    today = datetime.date.today().strftime("%Y-%m-%d")
    filtered_csv = f"finviz_screeners_filtered_{today}.csv"
    filtered_html = f"finviz_screeners_filtered_{today}.html"
    filtered_df.to_csv(filtered_csv, index=False)
    filtered_df.to_html(filtered_html, index=False)
    print(f"Filtered CSV: {filtered_csv}\nFiltered HTML: {filtered_html}")

    # 4. Generate gallery for filtered tickers
    gallery_path = generate_finviz_gallery(filtered_df['Ticker'].tolist())
    print(f"Filtered chart gallery HTML: {gallery_path}")


10% Change: 13 tickers found
Growth: 34 tickers found
IPO: 46 tickers found
52 Week High: 11 tickers found
Week 20%+ Gain: 37 tickers found
Total tickers before ATR% filter: 125
Rate limited fetching ATR% for ARM; retrying in 1s...
Rate limited fetching ATR% for CPRX; retrying in 1s...
Rate limited fetching ATR% for GDOT; retrying in 1s...
Rate limited fetching ATR% for MRP; retrying in 1s...
Rate limited fetching ATR% for SE; retrying in 1s...
Rate limited fetching ATR% for TACOU; retrying in 1s...
Rate limited fetching ATR% for VIK; retrying in 1s...
Tickers with ATR% > 3%: 0
Filtered CSV: finviz_screeners_filtered_2025-05-11.csv
Filtered HTML: finviz_screeners_filtered_2025-05-11.html
Filtered chart gallery HTML: finviz_chart_grid_2025-05-11.html
CPU times: user 19.5 s, sys: 137 ms, total: 19.6 s
Wall time: 51.3 s


In [ ]:
%%time

import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time

# ----------------------------
# Constants & Shared Session
# ----------------------------
FINVIZ_BASE = "https://finviz.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}

session = requests.Session()
session.headers.update(HEADERS)

# ----------------------------
# Part 1: Screener Fetch & Save
# ----------------------------

# Screener URLs
screener_urls = {
    "10% Change": (
        f"{FINVIZ_BASE}/screener.ashx?v=151"
        f"&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,"
        f"ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&"
        f"c=0,1,2,3,4,5,6,64,67,65,66"
    ),
    "Growth": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,"
        f"ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,"
        f"ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4"
    ),
    "IPO": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,"
        f"ta_sma20_pa&ft=4"
    ),
    "52 Week High": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4"
    ),
    "Week 20%+ Gain": (
        f"{FINVIZ_BASE}/screener.ashx?v=111"
        f"&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,"
        f"ta_volatility_wo4&ft=4&o=-marketcap&r=25"
    )
}

def fetch_all_tickers(screener_url: str, max_pages: int = 10) -> pd.DataFrame:
    """
    Fetch tickers from a Finviz screener URL, paginating up to max_pages.
    Returns DataFrame with columns: No., Ticker, Company, Sector, Industry,
    Country, Market Cap, P/E, Volume, Price, Change.
    """
    combined = []
    seen = set()
    page = 1

    while page <= max_pages:
        resp = session.get(f"{screener_url}&r={1+(page-1)*20}", timeout=10)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        rows = soup.select('tr[valign="top"]')
        if not rows:
            break

        new_data = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 11:
                ticker = cols[1].text.strip()
                if ticker and ticker not in seen:
                    combined.append([c.text.strip() for c in cols])
                    seen.add(ticker)
                    new_data = True
        if not new_data:
            break
        page += 1
        time.sleep(1)

    columns = ['No.', 'Ticker', 'Company', 'Sector', 'Industry',
               'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change']
    return pd.DataFrame(combined, columns=columns) if combined else pd.DataFrame(columns=columns)


def aggregate_and_save(screener_map: dict) -> (pd.DataFrame, str, str):
    """
    Aggregates tickers across screeners, saves CSV & HTML summary.
    Returns (DataFrame, csv_path, html_path).
    """
    mapping = defaultdict(list)
    today = datetime.date.today().strftime("%Y-%m-%d")

    for name, url in screener_map.items():
        df = fetch_all_tickers(url)
        print(f"{name}: {len(df)} tickers found")
        for t in df['Ticker'].unique():
            mapping[t].append(name)

    if not mapping:
        mapping['TSLA'].append('Default Screener')

    data = []
    for t, screens in mapping.items():
        data.append({
            'Ticker': t,
            'Appearances': len(screens),
            'Screeners': ", ".join(screens)
        })
    summary_df = pd.DataFrame(data).sort_values(['Appearances','Ticker'], ascending=[False, True])

    csv_file = f"finviz_screeners_{today}.csv"
    html_file = f"finviz_screeners_{today}.html"
    summary_df.to_csv(csv_file, index=False)
    summary_df.to_html(html_file, index=False)

    return summary_df, csv_file, html_file

# ----------------------------
# Part 2: ATR% Scraper & Filter
# ----------------------------

def get_atr_pct(ticker: str) -> float:
    """
    Scrapes Finviz for ATR (14) and Prev Close, returns ATR% or None.
    """
    resp = session.get(f"{FINVIZ_BASE}/quote.ashx", params={"t": ticker})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.content, "html.parser")
    table = soup.find("table", class_="snapshot-table2")
    if not table:
        return None
    data = {}
    for row in table.find_all("tr"):
        cells = row.find_all("td")
        for key_cell, val_cell in zip(cells[0::2], cells[1::2]):
            data[key_cell.get_text(strip=True)] = val_cell.get_text(strip=True)
    atr = data.get("ATR")
    prev = data.get("Prev Close")
    try:
        return (float(atr) / float(prev)) * 100
    except:
        return None

# ----------------------------
# Part 3: Chart Gallery
# ----------------------------

def generate_finviz_gallery(tickers: list) -> str:
    """
    Creates HTML displaying a 4-col grid of Finviz charts, dated filename.
    """
    today = datetime.date.today().strftime("%Y-%m-%d")
    out_html = f"finviz_chart_grid_{today}.html"
    html = [
        '<html><head><title>Finviz Chart Gallery</title>',
        '<style>',
        'body { font-family: Arial; background: #f5f5f5; padding: 20px }',
        '.chart-grid { display: flex; flex-wrap: wrap; gap: 20px }',
        '.chart-item { width: 23%; background: white; border:1px solid #ccc;',
        'padding:10px; box-shadow:2px 2px 5px rgba(0,0,0,0.1); text-align:center }',
        '.chart-item img { max-width:100%; height:auto }',
        'h2 { text-align:center }',
        '</style></head><body><h2>Finviz Chart Gallery</h2>',
        '<div class="chart-grid">'
    ]
    for t in tickers:
        params = {'t': t, 'ty': 'c', 'ta': '1', 'p': 'd', 's': 'm'}
        img_src = requests.Request('GET', f"{FINVIZ_BASE}/chart.ashx", params=params).prepare().url
        html.append(f'<div class="chart-item"><h4>{t}</h4>'
                    f'<img src="{img_src}" alt="{t}"></div>')
    html.append('</div></body></html>')
    with open(out_html, 'w') as f:
        f.write("\n".join(html))
    return out_html

# ----------------------------
# Main Execution
# ----------------------------
if __name__ == "__main__":
    # 1. Aggregate screeners
    summary_df, csv_path, html_summary = aggregate_and_save(screener_urls)
    print(f"Total tickers before ATR% filter: {len(summary_df)}")

    # 2. Compute ATR% and filter >3%
    summary_df['ATR%'] = summary_df['Ticker'].apply(get_atr_pct)
    filtered_df = summary_df[summary_df['ATR%'] > 3.0]
    print(f"Tickers with ATR% > 3%: {len(filtered_df)}")

    # 3. Save filtered results
    today = datetime.date.today().strftime("%Y-%m-%d")
    filtered_csv = f"finviz_screeners_filtered_{today}.csv"
    filtered_html = f"finviz_screeners_filtered_{today}.html"
    filtered_df.to_csv(filtered_csv, index=False)
    filtered_df.to_html(filtered_html, index=False)
    print(f"Filtered CSV: {filtered_csv}\nFiltered HTML: {filtered_html}")

    # 4. Generate gallery for filtered tickers
    gallery_path = generate_finviz_gallery(filtered_df['Ticker'].tolist())
    print(f"Filtered chart gallery HTML: {gallery_path}")


10% Change: 13 tickers found
Growth: 34 tickers found
IPO: 46 tickers found
52 Week High: 11 tickers found
Week 20%+ Gain: 37 tickers found
Total tickers before ATR% filter: 125


HTTPError: 429 Client Error: Too Many Requests for url: https://finviz.com/quote.ashx?t=ARM

In [ ]:
!pip install yfinance ta pandas beautifulsoup4

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ee45fdaf7ab428675bdc990939e8c4107b01207f370b01788f780a0a7c8f0598
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
from data_fetcher import fetch_data_from_url_with_pagination
from calculations import calculate_atr, calculate_atr_percentage, add_50_ma, calculate_percent_gain_from_50ma
from filters import filter_by_ma_atr_ratio



In [ ]:

# Define screener URLs
urls = {
#    "Screener1": "https://finviz.com/screener.ashx?v=151&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4"
    "Screener2": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_change_u10,ta_sma20_sa50,ta_sma50_pa&ft=4"
#    "Screener3": "https://finviz.com/screener.ashx?v=151&f=cap_smallover,sh_avgvol_o1000,sh_curvol_o100,ta_perf_13w50o,ta_volatility_mo5&ft=4&o=-marketcap&ar=180"
}

if __name__ == "__main__":
    # Store filtered results for each screener
    screener_results_atr = {}
    screener_results_ma_atr_ratio = {}

    for screener_name, url in urls.items():
        print(f"Fetching data from {screener_name}...")
        df = fetch_data_from_url_with_pagination(url)

        # Fetch tickers
        tickers = df['Ticker'].tolist()

        # Calculate ATR for each ticker
        # print(f"Calculating ATR for tickers in {screener_name}...")
        df['ATR'] = [calculate_atr(ticker, period='1mo', atr_window=14) for ticker in df['Ticker']]

        # Debug ATR column
        print(f"ATR values for {screener_name}:")
        print(df[['Ticker', 'ATR']])

        # Calculate ATR% for each ticker
        print(f"Calculating ATR% for tickers in {screener_name}...")
        df = calculate_atr_percentage(df)

        # Debug ATR% values
        print(f"ATR% values for {screener_name}:")
        print(df[['Ticker', 'ATR%', 'Price']])

        # # Add 50 MA to the DataFrame
        # print(f"Calculating 50 MA for {screener_name}...")
        # tickers = df['Ticker'].tolist()
        # df = add_50_ma(df, tickers)

        # # Calculate % Gain from 50 MA
        # print(f"Calculating % Gain from 50 MA for {screener_name}...")
        # df = calculate_percent_gain_from_50ma(df)

        # # Filter based on MA-ATR Ratio
        # print(f"Applying MA-ATR Ratio filter for {screener_name}...")
        # ma_atr_filtered_df = filter_by_ma_atr_ratio(df, threshold=1)

        # # Include relevant columns for output
        # ma_atr_filtered_df = ma_atr_filtered_df[
        #     ['Ticker', 'Price', 'ATR', 'ATR%', '% Gain from 50 MA', 'MA_ATR_Ratio']
        # ]

        # screener_results_ma_atr_ratio[screener_name] = ma_atr_filtered_df

        # print(f"MA-ATR Ratio Filtered results for {screener_name}:")
        # print(ma_atr_filtered_df)

    # Save filtered results to CSV
    for screener_name, atr_filtered_df in screener_results_atr.items():
        atr_filtered_df.to_csv(f"{screener_name}_atr_filtered.csv", index=False)

    # for screener_name, ma_atr_filtered_df in screener_results_ma_atr_ratio.items():
    #     ma_atr_filtered_df.to_csv(f"{screener_name}_ma_atr_filtered.csv", index=False)


Fetching data from Screener2...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Detected multi-index columns for AAOI. Normalizing...
Detected multi-index columns for CRDO. Normalizing...
Detected multi-index columns for IONQ. Normalizing...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Detected multi-index columns for MRVL. Normalizing...
Detected multi-index columns for NVCR. Normalizing...
Detected multi-index columns for SERV. Normalizing...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Detected multi-index columns for SOUN. Normalizing...
Detected multi-index columns for UMAC. Normalizing...
ATR values for Screener2:
  Ticker       ATR
0   AAOI  4.272582
1   CRDO  5.278336
2   IONQ  4.216305
3   MRVL  5.325973
4   NVCR  2.246982
5   SERV  1.352008
6   SOUN  1.475243
7   UMAC  2.461471
Calculating ATR% for tickers in Screener2...
0    4.272582
1    5.278336
2    4.216305
3    5.325973
4    2.246982
5    1.352008
6    1.475243
7    2.461471
Name: ATR, dtype: float64
0     36.43
1     75.95
2     33.83
3    120.77
4     33.41
5     13.08
6     16.91
7     11.10
Name: Price, dtype: float64
ATR% values for Screener2:
  Ticker       ATR%   Price
0   AAOI  11.728195   36.43
1   CRDO   6.949751   75.95
2   IONQ  12.463213   33.83
3   MRVL   4.410014  120.77
4   NVCR   6.725476   33.41
5   SERV  10.336453   13.08
6   SOUN   8.724088   16.91
7   UMAC  22.175419   11.10


In [ ]:
import importlib
import calculations  # Import your file

# Reload the file if changes are made
importlib.reload(calculations)


<module 'calculations' from '/content/calculations.py'>

In [ ]:
import os
os.getcwd()


'/content'

In [ ]:
os.listdir()

['.config',
 'data_fetcher.py',
 '__pycache__',
 'utils.py',
 'filters.py',
 'requirements.txt',
 'main.py',
 'calculations.py',
 'sample_data']

In [ ]:

# Define screener URLs
urls = {
#    "Screener1": "https://finviz.com/screener.ashx?v=151&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4"
    "Screener2": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_change_u10,ta_sma20_sa50,ta_sma50_pa&ft=4"
#    "Screener3": "https://finviz.com/screener.ashx?v=151&f=cap_smallover,sh_avgvol_o1000,sh_curvol_o100,ta_perf_13w50o,ta_volatility_mo5&ft=4&o=-marketcap&ar=180"
}

if __name__ == "__main__":
    # Store filtered results for each screener
    screener_results_atr = {}
    screener_results_ma_atr_ratio = {}

    for screener_name, url in urls.items():
        print(f"Fetching data from {screener_name}...")
        df = fetch_data_from_url_with_pagination(url)

        # Fetch tickers
        tickers = df['Ticker'].tolist()

        # Calculate ATR for each ticker
        # print(f"Calculating ATR for tickers in {screener_name}...")
        df['ATR'] = [calculate_atr(ticker, period='1mo', atr_window=14) for ticker in df['Ticker']]

        # Debug ATR column
        print(f"ATR values for {screener_name}:")
        print(df[['Ticker', 'ATR']])

        # Calculate ATR% for each ticker
        print(f"Calculating ATR% for tickers in {screener_name}...")
        df = calculate_atr_percentage(df)

        # Debug ATR% values
        print(f"ATR% values for {screener_name}:")
        print(df[['Ticker', 'ATR', 'Price', 'ATR%']])

        # # Add 50 MA to the DataFrame
        # print(f"Calculating 50 MA for {screener_name}...")
        # tickers = df['Ticker'].tolist()
        # df = add_50_ma(df, tickers)

        # # Calculate % Gain from 50 MA
        # print(f"Calculating % Gain from 50 MA for {screener_name}...")
        # df = calculate_percent_gain_from_50ma(df)

        # # Filter based on MA-ATR Ratio
        # print(f"Applying MA-ATR Ratio filter for {screener_name}...")
        # ma_atr_filtered_df = filter_by_ma_atr_ratio(df, threshold=1)

        # # Include relevant columns for output
        # ma_atr_filtered_df = ma_atr_filtered_df[
        #     ['Ticker', 'Price', 'ATR', 'ATR%', '% Gain from 50 MA', 'MA_ATR_Ratio']
        # ]

        # screener_results_ma_atr_ratio[screener_name] = ma_atr_filtered_df

        # print(f"MA-ATR Ratio Filtered results for {screener_name}:")
        # print(ma_atr_filtered_df)

    # Save filtered results to CSV
    for screener_name, atr_filtered_df in screener_results_atr.items():
        atr_filtered_df.to_csv(f"{screener_name}_atr_filtered.csv", index=False)

    # for screener_name, ma_atr_filtered_df in screener_results_ma_atr_ratio.items():
    #     ma_atr_filtered_df.to_csv(f"{screener_name}_ma_atr_filtered.csv", index=False)


Fetching data from Screener2...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Detected multi-index columns for AAOI. Normalizing...
Detected multi-index columns for CRDO. Normalizing...
Detected multi-index columns for IONQ. Normalizing...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Detected multi-index columns for MRVL. Normalizing...
Detected multi-index columns for NVCR. Normalizing...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Detected multi-index columns for SERV. Normalizing...
Detected multi-index columns for SOUN. Normalizing...
Detected multi-index columns for UMAC. Normalizing...
ATR values for Screener2:
  Ticker       ATR
0   AAOI  4.272582
1   CRDO  5.278336
2   IONQ  4.216305
3   MRVL  5.325973
4   NVCR  2.246982
5   SERV  1.352008
6   SOUN  1.475243
7   UMAC  2.461471
Calculating ATR% for tickers in Screener2...
ATR% values for Screener2:
  Ticker       ATR   Price       ATR%
0   AAOI  4.272582   36.43  11.728195
1   CRDO  5.278336   75.95   6.949751
2   IONQ  4.216305   33.83  12.463213
3   MRVL  5.325973  120.77   4.410014
4   NVCR  2.246982   33.41   6.725476
5   SERV  1.352008   13.08  10.336453
6   SOUN  1.475243   16.91   8.724088
7   UMAC  2.461471   11.10  22.175419


In [ ]:
import importlib
import calculations  # The file where you made changes

# Reload the module to reflect changes
importlib.reload(calculations)

# Import functions after reloading
from calculations import calculate_atr, calculate_atr_percentage


In [ ]:
H = 38.24
L = 33.56
C = 36.43

ATR = max(abs(H - L), abs(H - C), abs(L - C))

print(ATR)

4.68


In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time

In [ ]:


# Screener URLs
screener_urls = {
    "10% Change": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&c=0,1,2,3,4,5,6,64,67,65,66",
    "Growth": "https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4",
    "IPO": "https://finviz.com/screener.ashx?v=211&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,ta_sma20_pa&ft=4",
    "52 Week High": "https://finviz.com/screener.ashx?v=211&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4",
    "Week 20%+ Gain": "https://finviz.com/screener.ashx?v=211&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,ta_volatility_wo4&ft=4&o=-marketcap&r=25"
}

def fetch_all_tickers_from_screener(screener_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/91.0.4472.124 Safari/537.36"
    }
    session = requests.Session()
    session.headers.update(headers)

    combined_data = []
    page = 1

    while True:
        paged_url = f"{screener_url}&r={1 + (page - 1) * 20}"
        response = session.get(paged_url)

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('tr[valign="top"]')

        if not rows:
            break

        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 11:
                combined_data.append([cell.text.strip() for cell in cells])

        page += 1
        time.sleep(1)

    columns = [
        'No.', 'Ticker', 'Company', 'Sector', 'Industry',
        'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change'
    ]

    if combined_data:
        df = pd.DataFrame(combined_data, columns=columns)
    else:
        df = pd.DataFrame(columns=columns)

    return df

def aggregate_tickers(screener_urls):
    ticker_to_screeners = defaultdict(list)

    for name, url in screener_urls.items():
        df = fetch_all_tickers_from_screener(url)
        tickers = df['Ticker'].tolist()

        print(f"{name} fetched {len(tickers)} tickers.")

        for ticker in tickers:
            ticker_to_screeners[ticker].append(name)

    if not ticker_to_screeners:
        print("All screeners returned no results. Adding default ticker TSLA.")
        ticker_to_screeners['TSLA'].append('Default Screener')

    return ticker_to_screeners

def save_results(ticker_to_screeners):
    today = datetime.date.today().strftime("%Y-%m-%d")
    all_data = []

    for ticker, screeners in ticker_to_screeners.items():
        all_data.append({
            "Ticker": ticker,
            "Appearances": len(screeners),
            "Screeners": ", ".join(screeners)
        })

    df = pd.DataFrame(all_data)
    df.sort_values(by=["Appearances", "Ticker"], ascending=[False, True], inplace=True)

    filename = f"finviz_screeners_{today}.csv"
    df.to_csv(filename, index=False)
    print(f"\nResults saved to {filename}")

def display_results(ticker_to_screeners):
    print("\n=== All Tickers Grouped by Screener ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        print(f"{ticker}: from {', '.join(screeners)}")

    print("\n=== Strong Overlap Candidates (Appearing in Multiple Screeners) ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        if len(screeners) > 1:
            print(f"{ticker}: Appears in {len(screeners)} screeners -> {', '.join(screeners)}")

if __name__ == "__main__":
    ticker_to_screeners = aggregate_tickers(screener_urls)
    display_results(ticker_to_screeners)
    save_results(ticker_to_screeners)


KeyboardInterrupt: 

In [ ]:

# Screener URLs
screener_urls = {
    "10% Change": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&c=0,1,2,3,4,5,6,64,67,65,66",
    "Growth": "https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4",
    "IPO": "https://finviz.com/screener.ashx?v=211&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,ta_sma20_pa&ft=4",
    "52 Week High": "https://finviz.com/screener.ashx?v=211&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4",
    "Week 20%+ Gain": "https://finviz.com/screener.ashx?v=211&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,ta_volatility_wo4&ft=4&o=-marketcap&r=25"
}

def fetch_all_tickers_from_screener(screener_url, max_pages=10):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/91.0.4472.124 Safari/537.36"
    }
    session = requests.Session()
    session.headers.update(headers)

    combined_data = []
    page = 1

    while page <= max_pages:
        paged_url = f"{screener_url}&r={1 + (page - 1) * 20}"
        try:
            response = session.get(paged_url, timeout=10)
        except requests.exceptions.Timeout:
            print(f"Timeout fetching page {page}. Skipping...")
            break
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('tr[valign="top"]')

        if not rows:
            print(f"No rows found on page {page}. Ending pagination.")
            break

        page_has_valid_data = False
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 11:
                combined_data.append([cell.text.strip() for cell in cells])
                page_has_valid_data = True

        if not page_has_valid_data:
            print(f"No valid stock entries found on page {page}. Ending pagination.")
            break

        page += 1
        time.sleep(1)

    columns = [
        'No.', 'Ticker', 'Company', 'Sector', 'Industry',
        'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change'
    ]

    if combined_data:
        df = pd.DataFrame(combined_data, columns=columns)
    else:
        df = pd.DataFrame(columns=columns)

    return df

def aggregate_tickers(screener_urls):
    ticker_to_screeners = defaultdict(list)

    for name, url in screener_urls.items():
        df = fetch_all_tickers_from_screener(url)
        tickers = df['Ticker'].tolist()

        print(f"{name} fetched {len(tickers)} tickers.")

        for ticker in tickers:
            ticker_to_screeners[ticker].append(name)

    if not ticker_to_screeners:
        print("All screeners returned no results. Adding default ticker TSLA.")
        ticker_to_screeners['TSLA'].append('Default Screener')

    return ticker_to_screeners

def save_results(ticker_to_screeners):
    today = datetime.date.today().strftime("%Y-%m-%d")
    all_data = []

    for ticker, screeners in ticker_to_screeners.items():
        all_data.append({
            "Ticker": ticker,
            "Appearances": len(screeners),
            "Screeners": ", ".join(screeners)
        })

    df = pd.DataFrame(all_data)
    df.sort_values(by=["Appearances", "Ticker"], ascending=[False, True], inplace=True)

    filename = f"finviz_screeners_{today}.csv"
    df.to_csv(filename, index=False)
    print(f"\nResults saved to {filename}")

def display_results(ticker_to_screeners):
    print("\n=== All Tickers Grouped by Screener ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        print(f"{ticker}: from {', '.join(screeners)}")

    print("\n=== Strong Overlap Candidates (Appearing in Multiple Screeners) ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        if len(screeners) > 1:
            print(f"{ticker}: Appears in {len(screeners)} screeners -> {', '.join(screeners)}")

if __name__ == "__main__":
    ticker_to_screeners = aggregate_tickers(screener_urls)
    display_results(ticker_to_screeners)
    save_results(ticker_to_screeners)

10% Change fetched 11 tickers.
Growth fetched 27 tickers.
No rows found on page 1. Ending pagination.
IPO fetched 0 tickers.
No rows found on page 1. Ending pagination.
52 Week High fetched 0 tickers.
No rows found on page 1. Ending pagination.
Week 20%+ Gain fetched 0 tickers.

=== All Tickers Grouped by Screener ===

UPXI: from 10% Change
EPWK: from 10% Change, 10% Change, 10% Change, 10% Change, 10% Change, 10% Change, 10% Change, 10% Change, 10% Change, 10% Change
ADMA: from Growth
AEM: from Growth
AGI: from Growth
ATEC: from Growth
CI: from Growth
CNK: from Growth
EAT: from Growth
GENI: from Growth
GH: from Growth
GRAL: from Growth
KGC: from Growth
LLY: from Growth
NEM: from Growth
TGTX: from Growth
UBER: from Growth
WGS: from Growth
WTRG: from Growth
ZS: from Growth, Growth, Growth, Growth, Growth, Growth, Growth, Growth, Growth, Growth

=== Strong Overlap Candidates (Appearing in Multiple Screeners) ===

EPWK: Appears in 10 screeners -> 10% Change, 10% Change, 10% Change, 10% Ch

In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time

# Screener URLs
screener_urls = {
    "10% Change": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&c=0,1,2,3,4,5,6,64,67,65,66",
    "Growth": "https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4",
    "IPO": "https://finviz.com/screener.ashx?v=111&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,ta_sma20_pa&ft=4",
    "52 Week High": "https://finviz.com/screener.ashx?v=111&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4",
    "Week 20%+ Gain": "https://finviz.com/screener.ashx?v=111&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,ta_volatility_wo4&ft=4&o=-marketcap&r=25"
}

def fetch_all_tickers_from_screener(screener_url, max_pages=10):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/91.0.4472.124 Safari/537.36"
    }
    session = requests.Session()
    session.headers.update(headers)

    combined_data = []
    seen_tickers = set()
    page = 1

    while page <= max_pages:
        paged_url = f"{screener_url}&r={1 + (page - 1) * 20}"
        try:
            response = session.get(paged_url, timeout=10)
        except requests.exceptions.Timeout:
            print(f"Timeout fetching page {page}. Skipping...")
            break
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('tr[valign="top"]')

        if not rows:
            print(f"No rows found on page {page}. Ending pagination.")
            break

        page_new_data = False
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 11:
                ticker = cells[1].text.strip()
                if ticker and ticker not in seen_tickers:
                    combined_data.append([cell.text.strip() for cell in cells])
                    seen_tickers.add(ticker)
                    page_new_data = True

        if not page_new_data:
            print(f"No new tickers found on page {page}. Ending pagination.")
            break

        page += 1
        time.sleep(1)

    columns = [
        'No.', 'Ticker', 'Company', 'Sector', 'Industry',
        'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change'
    ]

    if combined_data:
        df = pd.DataFrame(combined_data, columns=columns)
    else:
        df = pd.DataFrame(columns=columns)

    return df

def aggregate_tickers(screener_urls):
    ticker_to_screeners = defaultdict(list)

    for name, url in screener_urls.items():
        df = fetch_all_tickers_from_screener(url)
        tickers = df['Ticker'].tolist()

        print(f"{name} fetched {len(tickers)} tickers.")

        for ticker in set(tickers):
            ticker_to_screeners[ticker].append(name)

    if not ticker_to_screeners:
        print("All screeners returned no results. Adding default ticker TSLA.")
        ticker_to_screeners['TSLA'].append('Default Screener')

    return ticker_to_screeners

def save_results(ticker_to_screeners):
    today = datetime.date.today().strftime("%Y-%m-%d")
    all_data = []

    for ticker, screeners in ticker_to_screeners.items():
        all_data.append({
            "Ticker": ticker,
            "Appearances": len(screeners),
            "Screeners": ", ".join(screeners)
        })

    df = pd.DataFrame(all_data)
    df.sort_values(by=["Appearances", "Ticker"], ascending=[False, True], inplace=True)

    filename = f"finviz_screeners_{today}.csv"
    df.to_csv(filename, index=False)
    print(f"\nResults saved to {filename}")

def display_results(ticker_to_screeners):
    print("\n=== All Tickers Grouped by Screener ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        print(f"{ticker}: from {', '.join(screeners)}")

    print("\n=== Strong Overlap Candidates (Appearing in Multiple Screeners) ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        if len(screeners) > 1:
            print(f"{ticker}: Appears in {len(screeners)} screeners -> {', '.join(screeners)}")

if __name__ == "__main__":
    ticker_to_screeners = aggregate_tickers(screener_urls)
    display_results(ticker_to_screeners)
    save_results(ticker_to_screeners)


No new tickers found on page 2. Ending pagination.
10% Change fetched 2 tickers.
No new tickers found on page 2. Ending pagination.
Growth fetched 18 tickers.
No new tickers found on page 3. Ending pagination.
IPO fetched 33 tickers.
No new tickers found on page 2. Ending pagination.
52 Week High fetched 6 tickers.
No new tickers found on page 3. Ending pagination.
Week 20%+ Gain fetched 32 tickers.

=== All Tickers Grouped by Screener ===

EPWK: from 10% Change, IPO
UPXI: from 10% Change
UBER: from Growth
GRAL: from Growth, IPO, Week 20%+ Gain
LLY: from Growth
KGC: from Growth
EAT: from Growth
CI: from Growth
ADMA: from Growth
WGS: from Growth
ZS: from Growth
AGI: from Growth
WTRG: from Growth
CNK: from Growth
TGTX: from Growth
NEM: from Growth
AEM: from Growth
GENI: from Growth
ATEC: from Growth
GH: from Growth
AHR: from IPO
TEM: from IPO, Week 20%+ Gain
BIRK: from IPO
MRP: from IPO
FATN: from IPO
TLN: from IPO
KRMN: from IPO
GEV: from IPO
USAR: from IPO
KVUE: from IPO
VIK: from IPO


In [ ]:
%%time

import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time

date = datetime.date.today().strftime("%Y-%m-%d")
print(date)

# Screener URLs
screener_urls = {
    "10% Change": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&c=0,1,2,3,4,5,6,64,67,65,66",
    "Growth": "https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4",
    "IPO": "https://finviz.com/screener.ashx?v=111&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,ta_sma20_pa&ft=4",
    "52 Week High": "https://finviz.com/screener.ashx?v=111&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4",
    "Week 20%+ Gain": "https://finviz.com/screener.ashx?v=111&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,ta_volatility_wo4&ft=4&o=-marketcap&r=25"
}

def fetch_all_tickers_from_screener(screener_url, max_pages=10):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/91.0.4472.124 Safari/537.36"
    }
    session = requests.Session()
    session.headers.update(headers)

    combined_data = []
    seen_tickers = set()
    page = 1

    while page <= max_pages:
        paged_url = f"{screener_url}&r={1 + (page - 1) * 20}"
        try:
            response = session.get(paged_url, timeout=10)
        except requests.exceptions.Timeout:
            print(f"Timeout fetching page {page}. Skipping...")
            break
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('tr[valign="top"]')

        if not rows:
            print(f"No rows found on page {page}. Ending pagination.")
            break

        page_new_data = False
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 11:
                ticker = cells[1].text.strip()
                if ticker and ticker not in seen_tickers:
                    combined_data.append([cell.text.strip() for cell in cells])
                    seen_tickers.add(ticker)
                    page_new_data = True

        if not page_new_data:
            print(f"No new tickers found on page {page}. Ending pagination.")
            break

        page += 1
        time.sleep(1)

    columns = [
        'No.', 'Ticker', 'Company', 'Sector', 'Industry',
        'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change'
    ]

    if combined_data:
        df = pd.DataFrame(combined_data, columns=columns)
    else:
        df = pd.DataFrame(columns=columns)

    return df

def aggregate_tickers(screener_urls):
    ticker_to_screeners = defaultdict(list)

    for name, url in screener_urls.items():
        df = fetch_all_tickers_from_screener(url)
        tickers = df['Ticker'].tolist()

        print(f"{name} fetched {len(tickers)} tickers.")

        for ticker in set(tickers):
            ticker_to_screeners[ticker].append(name)

    if not ticker_to_screeners:
        print("All screeners returned no results. Adding default ticker TSLA.")
        ticker_to_screeners['TSLA'].append('Default Screener')

    return ticker_to_screeners

def save_results(ticker_to_screeners):
    today = datetime.date.today().strftime("%Y-%m-%d")
    all_data = []

    for ticker, screeners in ticker_to_screeners.items():
        all_data.append({
            "Ticker": ticker,
            "Appearances": len(screeners),
            "Screeners": ", ".join(screeners)
        })

    df = pd.DataFrame(all_data)
    df.sort_values(by=["Appearances", "Ticker"], ascending=[False, True], inplace=True)

    filename = f"finviz_screeners_{today}.csv"
    df.to_csv(filename, index=False)
    print(f"\nResults saved to {filename}")

def display_results(ticker_to_screeners):
    print("\n=== All Tickers Grouped by Screener ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        print(f"{ticker}: from {', '.join(screeners)}")

    print("\n=== Strong Overlap Candidates (Appearing in Multiple Screeners) ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        if len(screeners) > 1:
            print(f"{ticker}: Appears in {len(screeners)} screeners -> {', '.join(screeners)}")

if __name__ == "__main__":
    ticker_to_screeners = aggregate_tickers(screener_urls)
    display_results(ticker_to_screeners)
    save_results(ticker_to_screeners)


2025-05-04
No new tickers found on page 2. Ending pagination.
10% Change fetched 8 tickers.
No new tickers found on page 3. Ending pagination.
Growth fetched 23 tickers.
No new tickers found on page 4. Ending pagination.
IPO fetched 43 tickers.
No new tickers found on page 2. Ending pagination.
52 Week High fetched 13 tickers.
No new tickers found on page 3. Ending pagination.
Week 20%+ Gain fetched 23 tickers.

=== All Tickers Grouped by Screener ===

IONQ: from 10% Change
DUOL: from 10% Change
UPXI: from 10% Change
TNXP: from 10% Change
BULL: from 10% Change
PRDO: from 10% Change
TRUP: from 10% Change
QNTM: from 10% Change
ZS: from Growth, 52 Week High
CI: from Growth
CNK: from Growth
HSAI: from Growth
VIRT: from Growth
ADMA: from Growth
SE: from Growth
UBER: from Growth
PWR: from Growth
BSX: from Growth
KGC: from Growth
BRBR: from Growth
GENI: from Growth
TVTX: from Growth
KC: from Growth, Week 20%+ Gain
GRAL: from Growth, IPO
CPRX: from Growth
DASH: from Growth
WTRG: from Growth
RB

In [ ]:
%%time

import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import datetime
import time

date = datetime.date.today().strftime("%Y-%m-%d")
print(date)

# Screener URLs
screener_urls = {
    "10% Change": "https://finviz.com/screener.ashx?v=151&f=ind_stocksonly,sh_avgvol_o500,sh_price_o5,ta_changeopen_u10,ta_sma20_sa50,ta_sma50_pa&ft=4&o=-relativevolume&c=0,1,2,3,4,5,6,64,67,65,66",
    "Growth": "https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsqoq_o20,fa_salesqoq_o20,ind_stocksonly,sh_avgvol_o1000,sh_price_o10,ta_perf_4wup,ta_perf2_13wup,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4",
    "IPO": "https://finviz.com/screener.ashx?v=111&f=ind_stocksonly,ipodate_prev2yrs,sh_avgvol_o1000,sh_price_o10,ta_sma20_pa&ft=4",
    "52 Week High": "https://finviz.com/screener.ashx?v=111&f=sh_avgvol_o1000,sh_price_o10,ta_beta_o1,ta_highlow52w_nh&ft=4",
    "Week 20%+ Gain": "https://finviz.com/screener.ashx?v=111&f=cap_smallover,sh_avgvol_o1000,sh_price_o3,ta_perf_1w20o,ta_volatility_wo4&ft=4&o=-marketcap&r=25"
}

def fetch_all_tickers_from_screener(screener_url, max_pages=10):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/91.0.4472.124 Safari/537.36"
    }
    session = requests.Session()
    session.headers.update(headers)

    combined_data = []
    seen_tickers = set()
    page = 1

    while page <= max_pages:
        paged_url = f"{screener_url}&r={1 + (page - 1) * 20}"
        try:
            response = session.get(paged_url, timeout=10)
        except requests.exceptions.Timeout:
            print(f"Timeout fetching page {page}. Skipping...")
            break
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('tr[valign="top"]')

        if not rows:
            print(f"No rows found on page {page}. Ending pagination.")
            break

        page_new_data = False
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 11:
                ticker = cells[1].text.strip()
                if ticker and ticker not in seen_tickers:
                    combined_data.append([cell.text.strip() for cell in cells])
                    seen_tickers.add(ticker)
                    page_new_data = True

        if not page_new_data:
            print(f"No new tickers found on page {page}. Ending pagination.")
            break

        page += 1
        time.sleep(1)

    columns = [
        'No.', 'Ticker', 'Company', 'Sector', 'Industry',
        'Country', 'Market Cap', 'P/E', 'Volume', 'Price', 'Change'
    ]

    if combined_data:
        df = pd.DataFrame(combined_data, columns=columns)
    else:
        df = pd.DataFrame(columns=columns)

    return df

def aggregate_tickers(screener_urls):
    ticker_to_screeners = defaultdict(list)

    for name, url in screener_urls.items():
        df = fetch_all_tickers_from_screener(url)
        tickers = df['Ticker'].tolist()

        print(f"{name} fetched {len(tickers)} tickers.")

        for ticker in set(tickers):
            ticker_to_screeners[ticker].append(name)

    if not ticker_to_screeners:
        print("All screeners returned no results. Adding default ticker TSLA.")
        ticker_to_screeners['TSLA'].append('Default Screener')

    return ticker_to_screeners

def save_results(ticker_to_screeners):
    today = datetime.date.today().strftime("%Y-%m-%d")
    all_data = []

    for ticker, screeners in ticker_to_screeners.items():
        all_data.append({
            "Ticker": ticker,
            "Appearances": len(screeners),
            "Screeners": ", ".join(screeners)
        })

    df = pd.DataFrame(all_data)
    df.sort_values(by=["Appearances", "Ticker"], ascending=[False, True], inplace=True)

    filename = f"finviz_screeners_{today}.csv"
    df.to_csv(filename, index=False)
    print(f"\nResults saved to {filename}")

def display_results(ticker_to_screeners):
    print("\n=== All Tickers Grouped by Screener ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        print(f"{ticker}: from {', '.join(screeners)}")

    print("\n=== Strong Overlap Candidates (Appearing in Multiple Screeners) ===\n")
    for ticker, screeners in ticker_to_screeners.items():
        if len(screeners) > 1:
            print(f"{ticker}: Appears in {len(screeners)} screeners -> {', '.join(screeners)}")

if __name__ == "__main__":
    ticker_to_screeners = aggregate_tickers(screener_urls)
    display_results(ticker_to_screeners)
    save_results(ticker_to_screeners)


2025-05-07
No new tickers found on page 2. Ending pagination.
10% Change fetched 9 tickers.
No new tickers found on page 3. Ending pagination.
Growth fetched 26 tickers.
No new tickers found on page 4. Ending pagination.
IPO fetched 42 tickers.
No new tickers found on page 2. Ending pagination.
52 Week High fetched 4 tickers.
No new tickers found on page 3. Ending pagination.
Week 20%+ Gain fetched 21 tickers.

=== All Tickers Grouped by Screener ===

AEVA: from 10% Change, Week 20%+ Gain
DFDV: from 10% Change
RGC: from 10% Change
TDUP: from 10% Change, Week 20%+ Gain
ULS: from 10% Change
IHS: from 10% Change
EPWK: from 10% Change
EPSM: from 10% Change
CGAU: from 10% Change
CTRE: from Growth
SE: from Growth
FUTU: from Growth
BSX: from Growth
DTE: from Growth
KC: from Growth, Week 20%+ Gain
WTRG: from Growth
CVNA: from Growth
UBER: from Growth
DUOL: from Growth, Week 20%+ Gain
EQT: from Growth
KGC: from Growth
EGO: from Growth
BCRX: from Growth, Week 20%+ Gain
AEM: from Growth
VIRT: fro